In [1]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
from PIL import Image, ImageEnhance
import random

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

def augment_image(image):
    """
    Apply random augmentation to the image (brightness, contrast, noise, etc.).
    :param image: Input image (numpy array in BGR format).
    :return: Augmented image (numpy array in BGR format).
    """
    # Convert to PIL Image for easier augmentation (PIL uses RGB format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = Image.fromarray(image)

    # Random brightness adjustment
    brightness_factor = random.uniform(0.8, 1.2)  # Adjust brightness randomly
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)

    # Random contrast adjustment
    contrast_factor = random.uniform(0.8, 1.2)  # Adjust contrast randomly
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)

    # Convert back to numpy array (RGB format)
    image = np.array(image)

    # Add Gaussian noise
    mean = 0
    var = random.uniform(0, 0.005)  # Random noise variance (reduced to avoid extreme changes)
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, image.shape).reshape(image.shape)
    image = image + gaussian * 255
    image = np.clip(image, 0, 255).astype(np.uint8)

    # Convert back to BGR format for OpenCV
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def apply_clahe(image):
    """
    Apply CLAHE to the image to enhance contrast.
    :param image: Input image (numpy array in BGR format).
    :return: Image with CLAHE applied (numpy array in BGR format).
    """
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge the channels back
    lab_clahe = cv2.merge((l_clahe, a, b))
    image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return image_clahe

def preprocess_image(image, apply_augmentation=True, apply_clahe_flag=True):
    """
    Preprocess the image with augmentation and CLAHE.
    :param image: Input image (numpy array in BGR format).
    :param apply_augmentation: Whether to apply augmentation.
    :param apply_clahe_flag: Whether to apply CLAHE.
    :return: Preprocessed image (numpy array in BGR format).
    """
    if apply_augmentation:
        image = augment_image(image)

    if apply_clahe_flag:
        image = apply_clahe(image)

    return image

def detect_fall(keypoints, threshold=0.5):
    """
    Detect fall based on keypoints.
    :param keypoints: Array of keypoints (17 keypoints, each with x, y, confidence).
    :param threshold: Confidence threshold for keypoints.
    :return: True if fall is detected, False otherwise.
    """
    # Indices for keypoints (COCO format)
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14

    # Get keypoints and confidence scores
    left_shoulder = keypoints[LEFT_SHOULDER * 3: (LEFT_SHOULDER + 1) * 3]
    right_shoulder = keypoints[RIGHT_SHOULDER * 3: (RIGHT_SHOULDER + 1) * 3]
    left_hip = keypoints[LEFT_HIP * 3: (LEFT_HIP + 1) * 3]
    right_hip = keypoints[RIGHT_HIP * 3: (RIGHT_HIP + 1) * 3]
    left_knee = keypoints[LEFT_KNEE * 3: (LEFT_KNEE + 1) * 3]
    right_knee = keypoints[RIGHT_KNEE * 3: (RIGHT_KNEE + 1) * 3]

    # Check confidence scores
    if (left_shoulder[2] < threshold or right_shoulder[2] < threshold or
        left_hip[2] < threshold or right_hip[2] < threshold or
        left_knee[2] < threshold or right_knee[2] < threshold):
        return False  # Skip if any keypoint is not confident

    # Calculate average y positions
    shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
    hip_y = (left_hip[1] + right_hip[1]) / 2
    knee_y = (left_knee[1] + right_knee[1]) / 2

    # Check if hip and knee are below shoulders (fall condition)
    if hip_y > shoulder_y and knee_y > shoulder_y:
        return True

    return False

# Real-time fall detection
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    preprocessed_frame = preprocess_image(frame, apply_augmentation=True, apply_clahe_flag=True)

    # Resize and normalize the frame for YOLOv7-pose
    image = letterbox(preprocessed_frame, 960, stride=64, auto=True)[0]
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))

    if torch.cuda.is_available():
        image = image.half().to(device)

    # Run inference
    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)

        # Detect fall
        if detect_fall(keypoints):
            cv2.putText(nimg, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the result
    cv2.imshow("Fall Detection", nimg)

    # Break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

cap.release()
cv2.destroyAllWindows()

C:\Users\LENOVO\anaconda3\envs\yolov7_env\lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [1]:
import torch
print(torch.__version__)

2.6.0+cu118


In [24]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
import math

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Constants from the journal
ALPHA = 0.5  # Adjustment factor for height threshold
SPEED_THRESHOLD = 0.5  # Vertical speed threshold for fall detection (pixels/frame)
ANGLE_THRESHOLD = 45  # Degrees threshold between torso and legs

# Previous frame data for speed calculation
prev_shoulder_y = None
frame_count = 0

def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def calculate_angle(a, b, c):
    """Calculate angle between points a, b, c in degrees"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def detect_fall(keypoints, threshold=0.5):
    global prev_shoulder_y, frame_count
    
    LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_KNEE, RIGHT_KNEE, LEFT_ANKLE, RIGHT_ANKLE = 5, 6, 11, 12, 13, 14, 15, 16
    
    keypoints_conf = [keypoints[i * 3: (i + 1) * 3] for i in [LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_KNEE, RIGHT_KNEE, LEFT_ANKLE, RIGHT_ANKLE]]
    
    # Check if keypoints are detected with sufficient confidence
    if any(kp[2] < threshold for kp in keypoints_conf[:4] + keypoints_conf[6:8]):
        return False
    
    # Current shoulder y position (average of left and right)
    current_shoulder_y = (keypoints_conf[0][1] + keypoints_conf[1][1]) / 2
    
    # Calculate vertical speed if we have previous data
    vertical_speed = 0
    if prev_shoulder_y is not None and frame_count > 0:
        vertical_speed = abs(current_shoulder_y - prev_shoulder_y) / frame_count
    
    prev_shoulder_y = current_shoulder_y
    frame_count = 0  # Reset counter after speed calculation
    
    # Calculate length factor (from shoulder to hip)
    length_factor = calculate_length_factor(keypoints_conf[0][:2], keypoints_conf[2][:2])
    
    # Check shoulder height relative to feet (with length factor adjustment)
    shoulder_low = (keypoints_conf[0][1] <= keypoints_conf[6][1] + ALPHA * length_factor and
                    keypoints_conf[1][1] <= keypoints_conf[7][1] + ALPHA * length_factor)
    
    # Calculate body dimensions
    body_height = abs(keypoints_conf[0][1] - keypoints_conf[6][1])
    body_width = abs(keypoints_conf[0][0] - keypoints_conf[1][0])
    horizontal_position = body_height < body_width
    
    # Calculate angle between hip, knee and ankle (for left and right legs)
    left_leg_angle = calculate_angle(keypoints_conf[2][:2], keypoints_conf[4][:2], keypoints_conf[6][:2])
    right_leg_angle = calculate_angle(keypoints_conf[3][:2], keypoints_conf[5][:2], keypoints_conf[7][:2])
    leg_angle = min(left_leg_angle, right_leg_angle)
    
    # Fall conditions from journal:
    # 1. Shoulders low relative to feet
    # 2. Body in horizontal position
    # 3. Either rapid movement (speed) or acute angle between torso and legs
    if shoulder_low and horizontal_position:
        if vertical_speed > SPEED_THRESHOLD or leg_angle < ANGLE_THRESHOLD:
            return True
    
    return False

# Path to video file
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    
    image = letterbox(frame, 640, stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))

    if torch.cuda.is_available():
        image = image.half().to(device)

    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)

        if detect_fall(keypoints):
            cv2.putText(nimg, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Fall Detection", nimg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

cap.release()
cv2.destroyAllWindows()
#IIni yg vid 1 nya worked ya

In [21]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
import math
import time

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Constants from the journal
ALPHA = 0.5  # Adjustment factor for height threshold
SPEED_THRESHOLD = 0.5  # Vertical speed threshold for fall detection (pixels/frame)
ANGLE_THRESHOLD = 45  # Degrees threshold between torso and legs
TARGET_FPS = 25  # Target frames per second for processing

# Previous frame data for speed calculation
prev_shoulder_y = None
frame_count = 0

def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def calculate_angle(a, b, c):
    """Calculate angle between points a, b, c in degrees"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def detect_fall(keypoints, threshold=0.5):
    global prev_shoulder_y, frame_count
    
    LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_KNEE, RIGHT_KNEE, LEFT_ANKLE, RIGHT_ANKLE = 5, 6, 11, 12, 13, 14, 15, 16
    
    keypoints_conf = [keypoints[i * 3: (i + 1) * 3] for i in [LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_KNEE, RIGHT_KNEE, LEFT_ANKLE, RIGHT_ANKLE]]
    
    # Check if keypoints are detected with sufficient confidence
    if any(kp[2] < threshold for kp in keypoints_conf[:4] + keypoints_conf[6:8]):
        return False
    
    # Current shoulder y position (average of left and right)
    current_shoulder_y = (keypoints_conf[0][1] + keypoints_conf[1][1]) / 2
    
    # Calculate vertical speed if we have previous data
    vertical_speed = 0
    if prev_shoulder_y is not None and frame_count > 0:
        vertical_speed = abs(current_shoulder_y - prev_shoulder_y) / frame_count
    
    prev_shoulder_y = current_shoulder_y
    frame_count = 0  # Reset counter after speed calculation
    
    # Calculate length factor (from shoulder to hip)
    length_factor = calculate_length_factor(keypoints_conf[0][:2], keypoints_conf[2][:2])
    
    # Check shoulder height relative to feet (with length factor adjustment)
    shoulder_low = (keypoints_conf[0][1] <= keypoints_conf[6][1] + ALPHA * length_factor and
                    keypoints_conf[1][1] <= keypoints_conf[7][1] + ALPHA * length_factor)
    
    # Calculate body dimensions
    body_height = abs(keypoints_conf[0][1] - keypoints_conf[6][1])
    body_width = abs(keypoints_conf[0][0] - keypoints_conf[1][0])
    horizontal_position = body_height < body_width
    
    # Calculate angle between hip, knee and ankle (for left and right legs)
    left_leg_angle = calculate_angle(keypoints_conf[2][:2], keypoints_conf[4][:2], keypoints_conf[6][:2])
    right_leg_angle = calculate_angle(keypoints_conf[3][:2], keypoints_conf[5][:2], keypoints_conf[7][:2])
    leg_angle = min(left_leg_angle, right_leg_angle)
    
    # Fall conditions from journal:
    # 1. Shoulders low relative to feet
    # 2. Body in horizontal position
    # 3. Either rapid movement (speed) or acute angle between torso and legs
    if shoulder_low and horizontal_position:
        if vertical_speed > SPEED_THRESHOLD or leg_angle < ANGLE_THRESHOLD:
            return True
    
    return False

# Path to video file
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (2).avi"
cap = cv2.VideoCapture(video_path)

# Get video properties
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Calculate skip frames to achieve 25 FPS
if original_fps > 0:
    skip_frames = max(1, int(round(original_fps / TARGET_FPS)))
else:
    skip_frames = 1  # Default if FPS info not available

print(f"Original FPS: {original_fps}, Processing at {TARGET_FPS} FPS, skipping {skip_frames-1} frames between processed frames")

frame_counter = 0
start_time = time.time()
processed_frames = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_counter += 1
    
    # Skip frames to achieve target FPS
    if frame_counter % skip_frames != 0:
        continue
    
    processed_frames += 1
    frame_count += 1
    
    image = letterbox(frame, 640, stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))

    if torch.cuda.is_available():
        image = image.half().to(device)

    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)

        if detect_fall(keypoints):
            cv2.putText(nimg, "Fall Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Calculate and display actual processing FPS
    elapsed_time = time.time() - start_time
    current_fps = processed_frames / elapsed_time if elapsed_time > 0 else 0
    cv2.putText(nimg, f"FPS: {current_fps:.1f}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    cv2.imshow("Fall Detection", nimg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Calculate final processing statistics
total_time = time.time() - start_time
actual_fps = processed_frames / total_time if total_time > 0 else 0
print(f"Processing complete. Actual FPS: {actual_fps:.2f}")

cap.release()
cv2.destroyAllWindows()

Original FPS: 25.0, Processing at 25 FPS, skipping 0 frames between processed frames
Processing complete. Actual FPS: 29.99


In [17]:
import numpy as np
import cv2
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model
import math
import time
# Update constants
SPEED_THRESHOLD = 100  # Pixels per second (contoh: 0.5 pixel/frame * 25 fps = 12.5 pixel/s)

# Track time between frames
prev_time = None
prev_shoulder_y = None

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Constants from the journal
ALPHA = 0.5  # Adjustment factor for height threshold
SPEED_THRESHOLD = 0.5  # Vertical speed threshold (pixels/frame)
ANGLE_THRESHOLD = 45  # Degrees threshold between torso and legs
TARGET_FPS = 25  # Target processing FPS

# Previous frame data for speed calculation
prev_shoulder_y = None
frame_count = 0

def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def calculate_angle(a, b, c):
    """Calculate angle between points a, b, c in degrees (b as vertex)"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1, 1))
    return np.degrees(angle)

def detect_fall(keypoints, threshold=0.5):
    global prev_shoulder_y, frame_count
    
    # Keypoint indices
    LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_ANKLE, RIGHT_ANKLE = 5, 6, 11, 12, 15, 16
    
    keypoints_conf = [keypoints[i * 3: (i + 1) * 3] for i in [LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_ANKLE, RIGHT_ANKLE]]
    
    # Check keypoint confidence
    if any(kp[2] < threshold for kp in keypoints_conf):
        return False
    
    # Calculate vertical speed (shoulder movement)
    current_shoulder_y = (keypoints_conf[0][1] + keypoints_conf[1][1]) / 2
    vertical_speed = 0
    if prev_shoulder_y is not None and frame_count > 0:
        vertical_speed = abs(current_shoulder_y - prev_shoulder_y) / frame_count
    prev_shoulder_y = current_shoulder_y
    frame_count = 0  # Reset counter
    
    # Length factor (shoulder to hip)
    length_factor = calculate_length_factor(keypoints_conf[0][:2], keypoints_conf[2][:2])
    
    # Shoulder and torso height check (relative to feet)
    shoulder_low = (keypoints_conf[0][1] <= keypoints_conf[4][1] + ALPHA * length_factor and
                    keypoints_conf[1][1] <= keypoints_conf[5][1] + ALPHA * length_factor)
    torso_low = (keypoints_conf[2][1] <= keypoints_conf[4][1] + ALPHA * length_factor and
                 keypoints_conf[3][1] <= keypoints_conf[5][1] + ALPHA * length_factor)
    
    # Body dimensions
    body_height = abs(keypoints_conf[0][1] - keypoints_conf[4][1])
    body_width = abs(keypoints_conf[0][0] - keypoints_conf[1][0])
    horizontal_position = body_height < body_width
    
    # Torso-leg angle calculation (at hip)
    left_angle = calculate_angle(keypoints_conf[0][:2], keypoints_conf[2][:2], keypoints_conf[4][:2])
    right_angle = calculate_angle(keypoints_conf[1][:2], keypoints_conf[3][:2], keypoints_conf[5][:2])
    torso_leg_angle = min(left_angle, right_angle)

    # Gunakan vertical_speed yang sudah dihitung di main loop
    if shoulder_low and torso_low and horizontal_position:
        if abs(vertical_speed) > SPEED_THRESHOLD or torso_leg_angle < ANGLE_THRESHOLD:
            return True
    return False
    
    # Fall conditions (all must be true)
    if shoulder_low and torso_low and horizontal_position:
        if vertical_speed > SPEED_THRESHOLD or torso_leg_angle < ANGLE_THRESHOLD:
            return True
    
    return False

# Video processing with FPS control
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
cap = cv2.VideoCapture(video_path)
original_fps = cap.get(cv2.CAP_PROP_FPS)
skip_frames = max(1, int(original_fps / TARGET_FPS))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % skip_frames != 0:
        continue

    current_time = time.perf_counter()

    # Hitung vertical speed
    if prev_shoulder_y is not None and prev_time is not None:
        time_diff = current_time - prev_time
        vertical_speed = (current_shoulder_y - prev_shoulder_y) / time_diff  # pixel/second
    else:
        vertical_speed = 0
    
    prev_shoulder_y = current_shoulder_y
    prev_time = current_time

    # Preprocess and inference
    image = letterbox(frame, 640, stride=64, auto=True)[0]
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))
    
    if torch.cuda.is_available():
        image = image.half().to(device)
    
    with torch.no_grad():
        output, _ = model(image)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)
    
    # Visualize results
    nimg = image[0].permute(1, 2, 0).cpu().numpy() * 255
    nimg = cv2.cvtColor(nimg.astype(np.uint8), cv2.COLOR_RGB2BGR)
    
    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)
        if detect_fall(keypoints, vertical_speed):
            cv2.putText(nimg, "FALL DETECTED!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.imshow("Fall Detection", nimg)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'current_shoulder_y' is not defined

In [26]:
import numpy as np
import cv2
import torch
import math
import time
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model

# Add the custom class to the safe globals list
torch.serialization.add_safe_globals([Model])

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Constants from the journal
ALPHA = 0.5          # Adjustment factor for height threshold
SPEED_THRESHOLD = 100 # Pixels per second (harus dikalibrasi)
ANGLE_THRESHOLD = 45 # Degrees threshold between torso and legs
TARGET_FPS = 25      # Target processing FPS

# Tracking variables
prev_shoulder_y = None
prev_time = None

def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def calculate_angle(a, b, c):
    """Calculate angle between points a, b, c in degrees (b as vertex)"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1, 1))
    return np.degrees(angle)

def detect_fall(keypoints, vertical_speed, threshold=0.5, debug_image=None):
    # Keypoint indices
    LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_ANKLE, RIGHT_ANKLE = 5, 6, 11, 12, 15, 16
    
    keypoints_conf = [keypoints[i * 3: (i + 1) * 3] for i in [LEFT_SHOULDER, RIGHT_SHOULDER, LEFT_HIP, RIGHT_HIP, LEFT_ANKLE, RIGHT_ANKLE]]
    
    # Debug: Gambar semua keypoint
    colors = [(0,255,0), (0,0,255), (255,0,0), (255,255,0), (0,255,255), (255,0,255)]
    for i, kp in enumerate(keypoints_conf):
        if kp[2] > threshold and debug_image is not None:
            x, y = int(kp[0]), int(kp[1])
            cv2.circle(debug_image, (x, y), 5, colors[i], -1)
    
    # Check keypoint confidence
    if any(kp[2] < threshold for kp in keypoints_conf):
        return False, "Keypoint confidence low"
    
    # Length factor (shoulder to hip)
    length_factor = calculate_length_factor(keypoints_conf[0][:2], keypoints_conf[2][:2])
    
    # Shoulder and torso height check (relative to feet)
    shoulder_threshold_left = keypoints_conf[4][1] + ALPHA * length_factor
    shoulder_threshold_right = keypoints_conf[5][1] + ALPHA * length_factor
    torso_threshold_left = keypoints_conf[4][1] + ALPHA * length_factor
    torso_threshold_right = keypoints_conf[5][1] + ALPHA * length_factor
    
    shoulder_low = (keypoints_conf[0][1] <= shoulder_threshold_left and
                    keypoints_conf[1][1] <= shoulder_threshold_right)
    torso_low = (keypoints_conf[2][1] <= torso_threshold_left and
                 keypoints_conf[3][1] <= torso_threshold_right)
    
    # Body dimensions
    body_height = abs(keypoints_conf[0][1] - keypoints_conf[4][1])
    body_width = abs(keypoints_conf[0][0] - keypoints_conf[1][0])
    horizontal_position = body_height < body_width
    
    # Torso-leg angle calculation (at hip)
    left_angle = calculate_angle(keypoints_conf[0][:2], keypoints_conf[2][:2], keypoints_conf[4][:2])
    right_angle = calculate_angle(keypoints_conf[1][:2], keypoints_conf[3][:2], keypoints_conf[5][:2])
    torso_leg_angle = min(left_angle, right_angle)
    
    # Debug: Gambar garis dan teks
    if debug_image is not None:
        # Gambar garis tinggi bahu
        y_shoulder_threshold = int((shoulder_threshold_left + shoulder_threshold_right)/2)
        cv2.line(debug_image, (0, y_shoulder_threshold), (debug_image.shape[1], y_shoulder_threshold), (0,255,0), 2)
        
        # Gambar sudut
        cv2.putText(debug_image, f"Angle: {torso_leg_angle:.1f}deg", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        cv2.putText(debug_image, f"Speed: {vertical_speed:.1f}px/s", (50, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        cv2.putText(debug_image, f"Body: W{body_width:.1f} > H{body_height:.1f}", (50, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
    
    # Fall conditions
    conditions = {
        "Shoulder Low": shoulder_low,
        "Torso Low": torso_low,
        "Horizontal Position": horizontal_position,
        "Speed/Angle": (abs(vertical_speed) > SPEED_THRESHOLD) or (torso_leg_angle < ANGLE_THRESHOLD)
    }
    
    if all([conditions["Shoulder Low"], conditions["Torso Low"], conditions["Horizontal Position"]]) and conditions["Speed/Angle"]:
        return True, "Fall detected"
    else:
        return False, conditions

# Video processing
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
cap = cv2.VideoCapture(video_path)

# Get video properties
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Frame skipping logic
skip_frames = max(1, int(original_fps / TARGET_FPS)) if original_fps > 0 else 1

# Initialize tracking
current_shoulder_y = None
vertical_speed = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Skip frames for target FPS
    if cap.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames != 0:
        continue
    
    # Preprocess
    image = letterbox(frame, 640, stride=64, auto=True)[0]
    image_tensor = transforms.ToTensor()(image)
    image_tensor = torch.tensor(np.array([image_tensor.numpy()]))
    
    if torch.cuda.is_available():
        image_tensor = image_tensor.half().to(device)
    
    # Inference
    with torch.no_grad():
        output, _ = model(image_tensor)
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)
    
    # Calculate vertical speed
    current_time = time.perf_counter()
    if output.shape[0] > 0:
        keypoints = output[0, 7:].T
        left_shoulder_y = keypoints[5*3+1]
        right_shoulder_y = keypoints[6*3+1]
        current_shoulder_y = (left_shoulder_y + right_shoulder_y) / 2
        
        if prev_shoulder_y is not None and prev_time is not None:
            time_diff = current_time - prev_time
            vertical_speed = (current_shoulder_y - prev_shoulder_y) / time_diff  # pixel/second
        else:
            vertical_speed = 0
        
        prev_shoulder_y = current_shoulder_y
        prev_time = current_time
    
    # Visualize
    nimg = image_tensor[0].permute(1, 2, 0).cpu().numpy() * 255
    nimg = cv2.cvtColor(nimg.astype(np.uint8), cv2.COLOR_RGB2BGR)
    
    for idx in range(output.shape[0]):
        keypoints = output[idx, 7:].T
        plot_skeleton_kpts(nimg, keypoints, 3)
        
        fall_detected, status = detect_fall(keypoints, vertical_speed, debug_image=nimg)
        
        if fall_detected:
            cv2.putText(nimg, "FALL DETECTED!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            # Tampilkan kondisi yang gagal
            y_offset = 250
            for cond, met in status.items():
                color = (0, 255, 0) if met else (0, 0, 255)
                text = f"{cond}: {'OK' if met else 'FAIL'}"
                cv2.putText(nimg, text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                y_offset += 30
    
    cv2.imshow("Fall Detection", nimg)
    
    # Show FPS
    fps = 1.0 / (time.perf_counter() - prev_time) if prev_time else 0
    cv2.putText(nimg, f"Speed: {vertical_speed:.1f} px/s", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.imshow("Fall Detection", nimg)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'str' object has no attribute 'items'

In [30]:
import numpy as np
import cv2
import torch
import math
import time
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model

# Inisialisasi device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model YOLOv7-pose
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# Parameter dari jurnal
ALPHA = 0.5          # Faktor penyesuaian tinggi
SPEED_THRESHOLD = 100 # Ambang kecepatan (pixel/detik)
ANGLE_THRESHOLD = 45  # Ambang sudut (derajat)
TARGET_FPS = 25       # Target FPS pemrosesan
KP_CONFIDENCE = 0.5   # Ambang keypoint confidence

# Variabel pelacakan
prev_shoulder_y = None
prev_time = None

def calculate_length_factor(shoulder, torso):
    """Menghitung faktor panjang antara bahu dan torso"""
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

def calculate_angle(a, b, c):
    """Menghitung sudut antara tiga titik (b sebagai vertex)"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1, 1))
    return np.degrees(angle)

def detect_fall(keypoints, vertical_speed, debug_image=None, threshold=0.5):
    # Keypoint indices sesuai jurnal
    LEFT_SHOULDER, RIGHT_SHOULDER = 5, 6
    LEFT_HIP, RIGHT_HIP = 11, 12
    LEFT_ANKLE, RIGHT_ANKLE = 15, 16

    # Ekstrak keypoint dengan confidence
    keypoints_dict = {
        'left_shoulder': keypoints[LEFT_SHOULDER*3:(LEFT_SHOULDER+1)*3],
        'right_shoulder': keypoints[RIGHT_SHOULDER*3:(RIGHT_SHOULDER+1)*3],
        'left_torso': keypoints[LEFT_HIP*3:(LEFT_HIP+1)*3],
        'right_torso': keypoints[RIGHT_HIP*3:(RIGHT_HIP+1)*3],
        'left_foot': keypoints[LEFT_ANKLE*3:(LEFT_ANKLE+1)*3],
        'right_foot': keypoints[RIGHT_ANKLE*3:(RIGHT_ANKLE+1)*3]
    }

    # Inisialisasi kondisi
    conditions = {
        'keypoints_valid': False,
        'shoulder_low': False,
        'torso_low': False,
        'horizontal_position': False,
        'speed_angle': False
    }
    message = ""

    # Cek confidence keypoint
    if any(kp[2] < threshold for kp in keypoints_dict.values()):
        return False, conditions, "Keypoint confidence low"
    
    conditions['keypoints_valid'] = True

    # 1. Hitung length factor
    L_factor = calculate_length_factor(
        keypoints_dict['left_shoulder'][:2], 
        keypoints_dict['left_torso'][:2]
    )

    # 2. Threshold tinggi
    shoulder_low = (
        (keypoints_dict['left_shoulder'][1] <= keypoints_dict['left_foot'][1] + ALPHA*L_factor) or
        (keypoints_dict['right_shoulder'][1] <= keypoints_dict['right_foot'][1] + ALPHA*L_factor)
    
    torso_low = (
        (keypoints_dict['left_torso'][1] <= keypoints_dict['left_foot'][1] + ALPHA*L_factor) or
        (keypoints_dict['right_torso'][1] <= keypoints_dict['right_foot'][1] + ALPHA*L_factor)
    
    conditions.update({
        'shoulder_low': shoulder_low,
        'torso_low': torso_low
    })

    # 3. Dimensi tubuh
    body_height = abs(
        (keypoints_dict['left_shoulder'][1] + keypoints_dict['right_shoulder'][1])/2 - 
        (keypoints_dict['left_foot'][1] + keypoints_dict['right_foot'][1])/2
    )
    body_width = abs(keypoints_dict['left_shoulder'][0] - keypoints_dict['right_shoulder'][0])
    horizontal_position = body_height < body_width
    conditions['horizontal_position'] = horizontal_position

    # 4. Sudut torso-kaki
    left_angle = calculate_angle(
        keypoints_dict['left_shoulder'][:2],
        keypoints_dict['left_torso'][:2],
        keypoints_dict['left_foot'][:2]
    )
    right_angle = calculate_angle(
        keypoints_dict['right_shoulder'][:2],
        keypoints_dict['right_torso'][:2],
        keypoints_dict['right_foot'][:2]
    )
    min_angle = min(left_angle, right_angle)
    speed_angle_condition = (abs(vertical_speed) > SPEED_THRESHOLD) or (min_angle < ANGLE_THRESHOLD)
    conditions['speed_angle'] = speed_angle_condition

    # 5. Keputusan akhir
    fall_conditions = [
        (shoulder_low or torso_low),
        horizontal_position,
        speed_angle_condition
    ]

    # Debug visualization
    if debug_image is not None:
        y_offset = 250
        for cond, met in conditions.items():
            color = (0, 255, 0) if met else (0, 0, 255)
            text = f"{cond}: {'OK' if met else 'FAIL'}"
            cv2.putText(debug_image, text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            y_offset += 30
        
        cv2.putText(debug_image, f"Speed: {vertical_speed:.1f} px/s", (50, y_offset+40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        cv2.putText(debug_image, f"Angle: {min_angle:.1f} deg", (50, y_offset+70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    if all(fall_conditions):
        return True, conditions, "Fall detected"
    return False, conditions, "Conditions not met"

def process_video(video_path):
    global prev_shoulder_y, prev_time
    
    cap = cv2.VideoCapture(video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    skip_frames = max(1, int(original_fps / TARGET_FPS)) if original_fps > 0 else 1
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Skip frames untuk target FPS
        if cap.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames != 0:
            continue

        # Preprocessing
        image = letterbox(frame, 640, stride=64, auto=True)[0]
        image_tensor = transforms.ToTensor()(image)
        image_tensor = torch.tensor(np.array([image_tensor.numpy()]))
        
        if torch.cuda.is_available():
            image_tensor = image_tensor.half().to(device)
        
        # Inference
        with torch.no_grad():
            output, _ = model(image_tensor)
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
        
        # Hitung kecepatan vertikal
        vertical_speed = 0
        current_time = time.perf_counter()
        
        if output.shape[0] > 0:
            keypoints = output[0, 7:].T
            left_shoulder = keypoints[5*3+1]
            right_shoulder = keypoints[6*3+1]
            current_shoulder_y = (left_shoulder + right_shoulder) / 2
            
            if prev_shoulder_y is not None and prev_time is not None:
                time_diff = current_time - prev_time
                vertical_speed = (current_shoulder_y - prev_shoulder_y) / time_diff
                
            prev_shoulder_y = current_shoulder_y
            prev_time = current_time
        
        # Visualisasi
        nimg = image_tensor[0].permute(1, 2, 0).cpu().numpy() * 255
        nimg = cv2.cvtColor(nimg.astype(np.uint8), cv2.COLOR_RGB2BGR)
        
        for idx in range(output.shape[0]):
            keypoints = output[idx, 7:].T
            plot_skeleton_kpts(nimg, keypoints, 3)
            
            # Deteksi jatuh dengan debug
            fall_detected, conditions, message = detect_fall(
                keypoints, 
                vertical_speed, 
                debug_image=nimg
            )
            
            # Tampilkan hasil
            if fall_detected:
                cv2.putText(nimg, "FALL DETECTED!", (50, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(nimg, message, (50, 100), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow("Fall Detection", nimg)
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Jalankan pemrosesan
video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
process_video(video_path)

SyntaxError: invalid syntax (823094185.py, line 90)